In [1]:
import os

import pandas as pd
import pandas_datareader as pdr
import yfinance as yf

## Configs

In [2]:
# save path
base_dir = "./data"
stock_path = os.path.join(base_dir, "stock_v241114.csv")
currency_path = os.path.join(base_dir, "currency_v241114.csv")
preproc_path = os.path.join(base_dir, "stock_currency_processed_v241114.csv")

os.makedirs(base_dir, exist_ok=True)

In [3]:
# set start/end date
start_date = "2010-01-01"
end_date = "2024-11-14"

# set ticker to be fetched
stock_tickers = ["SPY", "VOO", "QQQ", "SCHD"]

# set currency to be fetched
# source: https://fred.stlouisfed.org/
# e.g.
# - DEXKOUS: USD/Won (Korea)
ccy_tickers = ["DEXKOUS"]

# set index to be fetched
# source: https://fred.stlouisfed.org/
# e.g.
# - SP500: S&P 500
# - DJIA: Dow Jones Industrial Average
# - VIX: Volatility Index
# - NASDAQCOM: NASDAQ
# idx_tickers = ["SP500", "DJIA", "VIXCLS", "NASDAQCOM"]

## Fetch Stock / Currency / Index Data

In [4]:
data_stock = yf.download(
    stock_tickers,
    start=start_date,
    end=end_date,
    interval="1d",
)

data_currency = pdr.DataReader(
    ccy_tickers,
    data_source="fred",
    start=start_date,
    end=end_date,
)

# data_idx = pdr.DataReader(
#     idx_tickers,
#     data_source="fred",
#     start=start_date,
#     end=end_date,
# )

[*********************100%%**********************]  4 of 4 completed


In [5]:
data_stock.head()

Price       Adj Close                          Close                       \
Ticker            QQQ SCHD        SPY VOO        QQQ SCHD         SPY VOO   
Date                                                                        
2010-01-04  40.611572  NaN  86.315994 NaN  46.419998  NaN  113.330002 NaN   
2010-01-05  40.611572  NaN  86.544479 NaN  46.419998  NaN  113.629997 NaN   
2010-01-06  40.366623  NaN  86.605438 NaN  46.139999  NaN  113.709999 NaN   
2010-01-07  40.392853  NaN  86.970978 NaN  46.169998  NaN  114.190002 NaN   
2010-01-08  40.725292  NaN  87.260429 NaN  46.549999  NaN  114.570000 NaN   

Price            High       ...         Low           Open                   \
Ticker            QQQ SCHD  ...         SPY VOO        QQQ SCHD         SPY   
Date                        ...                                               
2010-01-04  46.490002  NaN  ...  111.510002 NaN  46.330002  NaN  112.370003   
2010-01-05  46.500000  NaN  ...  112.849998 NaN  46.389999  NaN  113.260002   
2010-01-06  46.549999  NaN  ...  113.430000 NaN  46.400002  NaN  113.519997   
2010-01-07  46.270000  NaN  ...  113.180000 NaN  46.209999  NaN  113.500000   
2010-01-08  46.549999  NaN  ...  113.660004 NaN  46.070000  NaN  113.889999   

Price             Volume                      
Ticker     VOO       QQQ SCHD        SPY VOO  
Date                                          
2010-01-04 NaN  62822800  NaN  118944600 NaN  
2010-01-05 NaN  62935600  NaN  111579900 NaN  
2010-01-06 NaN  96033000  NaN  116074400 NaN  
2010-01-07 NaN  77094100  NaN  131091100 NaN  
2010-01-08 NaN  88886600  NaN  126402800 NaN  

[5 rows x 24 columns]

In [6]:
data_stock.columns

MultiIndex([('Adj Close',  'QQQ'),
            ('Adj Close', 'SCHD'),
            ('Adj Close',  'SPY'),
            ('Adj Close',  'VOO'),
            (    'Close',  'QQQ'),
            (    'Close', 'SCHD'),
            (    'Close',  'SPY'),
            (    'Close',  'VOO'),
            (     'High',  'QQQ'),
            (     'High', 'SCHD'),
            (     'High',  'SPY'),
            (     'High',  'VOO'),
            (      'Low',  'QQQ'),
            (      'Low', 'SCHD'),
            (      'Low',  'SPY'),
            (      'Low',  'VOO'),
            (     'Open',  'QQQ'),
            (     'Open', 'SCHD'),
            (     'Open',  'SPY'),
            (     'Open',  'VOO'),
            (   'Volume',  'QQQ'),
            (   'Volume', 'SCHD'),
            (   'Volume',  'SPY'),
            (   'Volume',  'VOO')],
           names=['Price', 'Ticker'])

In [7]:
data_stock["Adj Close"].query("VOO > 0").head()

Ticker,QQQ,SCHD,SPY,VOO
Date,,,,
2010-09-09,40.820854,NaN,85.231926,77.970108
2010-09-10,40.970325,NaN,85.662262,78.324127
2010-09-13,41.541782,NaN,86.615082,79.309105
2010-09-14,41.717648,NaN,86.561310,79.293724
2010-09-15,41.981392,NaN,86.891685,79.493782


In [8]:
data_stock["Adj Close"].query("SCHD > 0").head()

Ticker,QQQ,SCHD,SPY,VOO
Date,,,,
2011-10-20,50.244564,7.038486,95.853622,87.715286
2011-10-21,50.874947,7.176552,97.673660,89.403648
2011-10-24,51.931507,7.215999,98.871185,90.539757
2011-10-25,50.910450,7.097657,96.948769,88.788277
2011-10-26,50.848305,7.170915,97.933632,89.640358


In [9]:
data_currency.head()

,DEXKOUS
DATE,
2010-01-01,NaN
2010-01-04,1149.0
2010-01-05,1138.2
2010-01-06,1130.0
2010-01-07,1135.0


## Preprocess Feched Data

In [10]:
data_stock_close = data_stock["Adj Close"]
data_stock_volume = data_stock["Volume"]

data_stock_close.columns = [f"{c}_close" for c in data_stock_close.columns]
data_stock_volume.columns = [f"{c}_volume" for c in data_stock_volume.columns]

In [11]:
data_stock_processed = data_stock_close.join(data_stock_volume)
data_stock_processed.index = pd.DatetimeIndex(data_stock_processed.index, name="date")

In [12]:
data_currency_processed = data_currency.copy()
data_currency_processed.columns = ["currency"]
data_currency_processed.index = pd.DatetimeIndex(data_currency_processed.index, name="date")

In [13]:
data_processed = data_stock_processed.join(data_currency_processed)

In [14]:
data_processed.head()

,QQQ_close,SCHD_close,SPY_close,VOO_close,QQQ_volume,SCHD_volume,SPY_volume,VOO_volume,currency
date,,,,,,,,,
2010-01-04,40.611572,NaN,86.315994,NaN,62822800,NaN,118944600,NaN,1149.00
2010-01-05,40.611572,NaN,86.544479,NaN,62935600,NaN,111579900,NaN,1138.20
2010-01-06,40.366623,NaN,86.605438,NaN,96033000,NaN,116074400,NaN,1130.00
2010-01-07,40.392853,NaN,86.970978,NaN,77094100,NaN,131091100,NaN,1135.00
2010-01-08,40.725292,NaN,87.260429,NaN,88886600,NaN,126402800,NaN,1127.75


## Save Results

In [15]:
data_stock.to_csv(stock_path)
data_currency.to_csv(currency_path)
data_processed.to_csv(preproc_path)